## Fine Tuning

In [ ]:
#Open API Fine Tuning Dashboard URL: https://platform.openai.com/finetune/

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# --- Load API Key ---
load_dotenv(override=True, dotenv_path="../.env.local")
my_api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=my_api_key)

In [2]:
with open("data/training_data.jsonl", "rb") as f:
    training_file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded training file ID:", training_file_obj.id)

with open("data/validation_data.jsonl", "rb") as f:
    validation_file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded validation file ID:", validation_file_obj.id)

Uploaded training file ID: file-WPVC3gbyG1eLafSY83wUCA
Uploaded validation file ID: file-PJdYutKyBD5rb9TjMvsWHc


In [ ]:
# Create a fine-tuning job
job = client.fine_tuning.jobs.create(
    training_file=training_file_obj.id,       # The file ID you uploaded
    validation_file=validation_file_obj.id,     # Optional
    model="gpt-4.1-nano-2025-04-14",               # or "gpt-3.5-turbo", etc.
    suffix="brand-customer-support"       # Optional model name suffix
)

print("Fine-tune job created:", job.id)

In [ ]:
#Get status of fine-tuning job
jobs = client.fine_tuning.jobs.list(limit=5)
for j in jobs.data:
    print(j.id, j.status, j.fine_tuned_model)

In [ ]:
def ask_question_without_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

def ask_question_with_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="ft:gpt-4.1-nano-2025-04-14:personal:brand-customer-support:D4TwzJgJ",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

In [5]:
import time
while True:
    # Ask user for a question
    user_prompt = input("Ask something: ")

    if (user_prompt.lower() != 'quit'):
        # Get and print the response
        response = ask_question_without_finetuning(user_prompt)
        print("\n[WITHOUT FINE TUNING] OpenAI says:", response)

        response = ask_question_with_finetuning(user_prompt)
        print("\n[WITH FINE TUNING] OpenAI says:", response)

        # add delay of 3 seconds
        time.sleep(3)
    else:
        break    

User asked: my glasses broke
ChatCompletion(id='chatcmpl-D4XnxHzd4lAK1cOMOAxRmGF6ZjTfE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='That’s frustrating—sorry about your glasses. I can help you fix it temporarily and point you to a repair.\n\nA few quick questions:\n- What part broke (frame hinge/temple, nose pad, or a lens)?\n- Do you have a spare pair or contacts to use in the meantime?\n- Where are you located? I can help you find a nearby repair shop.\n\nTemporary fix (for most common breaks):\n- If a temple/hinge is cracked: clean the area and place a small piece of clear tape on the outside to hold the pieces together. Avoid taping over the lenses.\n- If a lens is loose or popped out: press the lens back into the frame and use tape around the edge of the frame (not on the lens) to secure it temporarily.\n- If the frame is cracked near the hinge: you can use a tiny dab of super glue, but only on the frame ends (not on lenses)